## Data Class

In [34]:
import random

class Sentiment:
    neg = 'NEGATIVE'
    neu = 'NEUTRAL'
    pos = 'POSITIVE'
    
class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.neg
        elif self.score == 3:
            return Sentiment.neu
        else:
            return Sentiment.pos
        
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.neg, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.pos, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

## Load Data

In [35]:
import json

file_name = 'Books_small_10000.json'

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))
                       
reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

## Prep Data

In [54]:
from sklearn.model_selection import train_test_split

# Split the data into training and test, 77% train, 33% test
training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_cont = ReviewContainer(training)
test_cont = ReviewContainer(test)

len(test_cont.reviews)

3300

In [55]:
# Because there is a huge difference between container sizes, evaluations
# were not as accurate as they could be. Cutting sizes to be equal helped 
train_cont.evenly_distribute()
train_x = train_cont.get_text()
train_y = train_cont.get_sentiment()

test_cont.evenly_distribute()
test_x = test_cont.get_text()
test_y = test_cont.get_sentiment()

print(train_y.count(Sentiment.pos))
print(train_y.count(Sentiment.neg))
print(test_y.count(Sentiment.pos))
print(test_y.count(Sentiment.neg))

436
436
208
208


### Bag of words vectorization

In [56]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# count vectorizer basically counts the number of time each word comes up
# tfdif negates the ones that are always coming up such as "this, was, a"
vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)

## Classification
### Different methods for cross validation

#### Support Vector Machine

In [60]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

print(test_x[0])
# print(test_x_vectors[0])

clf_svm.predict(test_x_vectors[0])

# Here we can see that it was predicted accurately and 
# will go on testing with other classifiers

Not something I would read again.. Sorry but the stories just lack substance .. It's basically &#34;porn&#34; in written form.. Only thing missing is the cheesy music..


array(['NEGATIVE'], dtype='<U8')

#### Decision Tree

In [40]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

#### Naive bayes

In [41]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = DecisionTreeClassifier()
clf_gnb.fit(train_x_vectors, train_y)

clf_gnb.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

#### Logistic Regression

In [42]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

## Evaluation

In [43]:
# Mean accuracy
# Checks accuracy of the predictions compare with the results
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors, test_y))
print(clf_log.score(test_x_vectors, test_y))

0.8076923076923077
0.6274038461538461
0.6370192307692307
0.8052884615384616


In [44]:
# F1 Scores
from sklearn.metrics import f1_score
# pass in y_true y_predicted, optional labels
# Check accuracy for positive and negative reviews

print(f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.pos, Sentiment.neg]))
print(f1_score(test_y, clf_dec.predict(test_x_vectors), average=None, labels=[Sentiment.pos, Sentiment.neg]))
print(f1_score(test_y, clf_gnb.predict(test_x_vectors), average=None, labels=[Sentiment.pos, Sentiment.neg]))
print(f1_score(test_y, clf_log.predict(test_x_vectors), average=None, labels=[Sentiment.pos, Sentiment.neg]))

# SVM and Logistic regression works best for this dataset

[0.80582524 0.80952381]
[0.62102689 0.63356974]
[0.6234414  0.64965197]
[0.80291971 0.80760095]


In [62]:
test_set = ['I thoroughly enjoyed this, 5 stars', 'this is not a good buy', 'great pick up']

new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'POSITIVE'], dtype='<U8')

### Tuning the model

In [68]:
from sklearn.model_selection import GridSearchCV
# Check linear kernel with all the C values, figure which is the best option
# Does the same with rbf kernel, and then returns the optimal kernal and C val
parameters = {'kernel':('linear', 'rbf'), 'C':(1,4,8,16,32)}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x_vectors, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [66]:
print(clf.score(test_x_vectors, test_y))

0.8197115384615384


## Saving Model

In [49]:
import pickle
with open('./models/sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

In [50]:
# loaded_clf.predict(test_x_vectors) - wouldnt work before loading

In [51]:
with open('./models/sentiment_classifier.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

In [52]:
print(test_x[2])
loaded_clf.predict(test_x_vectors[2])

The Book Started Good but it Got Boring Quick It kept going in Circle till It lost me this is not a Easy Read I actually read It to put me to sleep after a half of a chapter you will be Zzzzzzzz Melatonin don't got nothing on this book.


array(['NEGATIVE'], dtype='<U8')